In [ ]:
import re
import pydicom
import os
import logging
import dicom2nifti

In [ ]:
img_path = '/nvmescratch/ceib/Prostate/Bounding_Boxes'

data = []  # List to collect the data

for subject in os.listdir(os.path.join(img_path, 'p0052021')):
    for session in os.listdir(os.path.join(img_path, 'p0052021', subject)):
        for image in os.listdir(os.path.join(img_path, 'p0052021', subject, session)):
            if image == 'sr' or image.startswith('SEG'):
                continue
            img = os.path.join('p0052021', subject, session, image,'DICOM')
            data.append({
                'subject': subject,
                'session': session,
                'image': img,
            })

# Create DataFrame from collected data
df_images = pd.DataFrame(data, columns=['subject', 'session', 'image'])

data = []  # List to collect the data

for subject in os.listdir(os.path.join(img_path, 'p0052021')):
    for session in os.listdir(os.path.join(img_path, 'p0052021', subject)):
        for image in os.listdir(os.path.join(img_path, 'p0052021', subject, session)):
            if image != 'sr':
                if image.startswith('SEG'):
                    bounding_box = os.path.join('p0052021', subject, session, image,'SEG')
                    image_associated = os.path.join('p0052021', subject, session,re.findall('\d+',image.split('_')[-1])[0],'DICOM')

                    data.append({
                        'subject': subject,
                        'session': session,
                        'image_associated': image_associated,
                        'bounding_box': bounding_box
                    })

# Create DataFrame from collected data
df_bounding_boxes = pd.DataFrame(data, columns=['subject', 'session', 'image_associated', 'bounding_box'])


merged_df = df_images.merge(df_bounding_boxes, 
                            left_on=['subject', 'session', 'image'], 
                            right_on=['subject', 'session', 'image_associated'], 
                            how='left')

In [ ]:
merged_df[~merged_df.image_associated.isna()]

,subject,session,image,image_associated,bounding_box
1,005586,006040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...
2,005586,006040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...
9,005596,006050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...
13,005700,006173,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/SEG_20240113_211959_757...
15,005700,006173,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/SEG_20240113_211927_274...
...,...,...,...,...,...
1310,005646,006108,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/SEG_20240113_210039_451...
1311,005646,006108,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/SEG_20240113_205921_383...
1312,005646,006108,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/SEG_20240113_205854_334...
1314,005399,005819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...


In [ ]:
os.mkdir(os.path.join(img_path, 'Prostate_Segmentation'))

In [ ]:
SeriesDescription = []
img_path = '/nvmescratch/ceib/Prostate/Bounding_Boxes'
for i, row in merged_df.iterrows():
    imgs = os.path.join(img_path, row['image'])
    try:
        files = os.listdir(imgs)
        for file in files:
            if file.endswith('.dcm'):
                ds = pydicom.dcmread(os.path.join(imgs, file))
                SeriesDescription.append(ds.SeriesDescription)
            break
    except:
        print(row['bounding_box'])
        SeriesDescription.append(None)

merged_df['SeriesDescription'] = SeriesDescription

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [ ]:
merged_df

,subject,session,image,image_associated,bounding_box,SeriesDescription
0,005586,006040,p0052021/005586/006040/9/DICOM,NaN,NaN,cor T2
1,005586,006040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2
2,005586,006040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS
3,005586,006040,p0052021/005586/006040/6/DICOM,NaN,NaN,sag T2 FRFSE
4,005586,006040,p0052021/005586/006040/5/DICOM,NaN,NaN,CORONAL STIR FSE
...,...,...,...,...,...,...
1322,005399,005819,p0052021/005399/005819/1101/DICOM,NaN,NaN,T1W_TSE_ax BARRIDO
1323,005399,005819,p0052021/005399/005819/1005/DICOM,NaN,NaN,sb 2000
1324,005399,005819,p0052021/005399/005819/1004/DICOM,NaN,NaN,ssb-800
1325,005399,005819,p0052021/005399/005819/1003/DICOM,NaN,NaN,dADC


In [ ]:
merged_df

,subject,session,image,image_associated,bounding_box,SeriesDescription
0,005586,006040,p0052021/005586/006040/9/DICOM,NaN,NaN,cor T2
1,005586,006040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2
2,005586,006040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS
3,005586,006040,p0052021/005586/006040/6/DICOM,NaN,NaN,sag T2 FRFSE
4,005586,006040,p0052021/005586/006040/5/DICOM,NaN,NaN,CORONAL STIR FSE
...,...,...,...,...,...,...
1322,005399,005819,p0052021/005399/005819/1101/DICOM,NaN,NaN,T1W_TSE_ax BARRIDO
1323,005399,005819,p0052021/005399/005819/1005/DICOM,NaN,NaN,sb 2000
1324,005399,005819,p0052021/005399/005819/1004/DICOM,NaN,NaN,ssb-800
1325,005399,005819,p0052021/005399/005819/1003/DICOM,NaN,NaN,dADC


In [ ]:
only_bounding = merged_df[~merged_df.bounding_box.isna()]

In [ ]:
only_bounding['image_nifti'] = only_bounding.image_associated.str.replace('DICOM', 'NIFTI')
only_bounding['bval'] = None
only_bounding['bvec'] = None
only_bounding['modality'] = None
no_nifti = []

def detect_modality(file_name):
    modality_patterns = {
        'T2': re.compile(r'T2', re.IGNORECASE),
        'ADC': re.compile(r'ADC', re.IGNORECASE),
        'DWI': re.compile(r'DW', re.IGNORECASE)
    }
    for modality, pattern in modality_patterns.items():
        if pattern.search(file_name):
            return modality
    return None

for i, row in only_bounding.iterrows():
    path = os.path.join(img_path, row['image_nifti'])
    if not os.path.exists(path):
        logging.warning(f"Path does not exist: {path}")
        no_nifti.append(row['image_nifti'])
        os.mkdir(path)
        dicom2nifti.convert_directory(os.path.join(img_path, row['image_associated']), path, compression=True)
        #continue

    try:
        imgs = os.listdir(path)
        for file in imgs:
            new_path = os.path.join(path, file)
            if file.endswith('.nii.gz'):
                only_bounding.at[i, 'image_nifti'] = new_path
            if file.endswith('bval'):
                only_bounding.at[i, 'bval'] = new_path
            if file.endswith('bvec'):
                only_bounding.at[i, 'bvec'] = new_path  
            modality = detect_modality(only_bounding.at[i, 'SeriesDescription'])
            if modality:
                only_bounding.at[i, 'modality'] = modality
    except Exception as e:
        logging.error(f"Error processing {path}: {e}")
        no_nifti.append(row['image_nifti'])

/tmp/ipykernel_227876/3910288657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_bounding['image_nifti'] = only_bounding.image_associated.str.replace('DICOM', 'NIFTI')
/tmp/ipykernel_227876/3910288657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  only_bounding['bval'] = None
/tmp/ipykernel_227876/3910288657.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [ ]:
only_bounding

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality
1,005586,006040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
2,005586,006040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
9,005596,006050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...,AXIAL T2 TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
13,005700,006173,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/9/DICOM,p0052021/005700/006173/SEG_20240113_211959_757...,AXIAL DW 1500 PROSTATA,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,DWI
15,005700,006173,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/7/DICOM,p0052021/005700/006173/SEG_20240113_211927_274...,AXIAL DW 1000 PROSTATA,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,DWI
...,...,...,...,...,...,...,...,...,...,...
1310,005646,006108,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/4/DICOM,p0052021/005646/006108/SEG_20240113_210039_451...,AXI PELVIS DWI B50-800,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,DWI
1311,005646,006108,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/1050/DICOM,p0052021/005646/006108/SEG_20240113_205921_383...,ADC (10^-6 mm²/s):Nov 26 2019 16-59-11 CET,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,ADC
1312,005646,006108,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/1000/DICOM,p0052021/005646/006108/SEG_20240113_205854_334...,DW_Synthetic: Ax FOCUS B-50-800 Synthetic B-2000,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,DWI
1314,005399,005819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...,T2W_TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2


In [ ]:
only_bounding[only_bounding.modality == 'T2']

,subject,session,image,image_associated,bounding_box,SeriesDescription,image_nifti,bval,bvec,modality
1,005586,006040,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/8/DICOM,p0052021/005586/006040/SEG_20240113_205302_952...,ax T2,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
2,005586,006040,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/7/DICOM,p0052021/005586/006040/SEG_20240113_205223_953...,Ax T2 ADENOPATIAS,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
9,005596,006050,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/13001/DICOM,p0052021/005596/006050/SEG_20240113_211223_114...,AXIAL T2 TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
16,005700,006173,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/6/DICOM,p0052021/005700/006173/SEG_20240113_211858_507...,AXIAL T2 ALTA RES. (de abajo arriba en próstata),/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
23,005665,006131,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/6/DICOM,p0052021/005665/006131/SEG_20240115_203626_074...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
...,...,...,...,...,...,...,...,...,...,...
1297,005636,006097,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/501/DICOM,p0052021/005636/006097/SEG_20240113_202733_764...,T2W_TSE_TRA_IQ,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
1301,005613,006070,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/6/DICOM,p0052021/005613/006070/SEG_20240113_203814_089...,AX T2 PROPELLER,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
1306,005646,006108,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/7/DICOM,p0052021/005646/006108/SEG_20240113_210319_135...,Ax T2 FSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2
1314,005399,005819,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/701/DICOM,p0052021/005399/005819/SEG_20240109_114231_571...,T2W_TSE,/nvmescratch/ceib/Prostate/Bounding_Boxes/p005...,None,None,T2


In [ ]:
only_bounding.to_csv('/home/jaalzate/Prostate_Cancer_TFM/Files/Boundingg_Boxes/Only_Bounding.csv', index=False)
merged_df.to_csv('/home/jaalzate/Prostate_Cancer_TFM/Files/Boundingg_Boxes/Merged_Bounding.csv', index=False)